This jupyter notebook is to show function `babs-init`.

In [1]:
# generate html file from this ipynb: 
# 1. set up the terminal:
#   $ source ~/.bashrc   # freesurfer and templateflow env variables
#   $ conda activate mydatalad
# 2. manually install BABS:
#   $ pip install -e .
# 3. then run:
#   $ jupyter nbconvert --execute --to html notebooks/show_babs_init_InputBIDS.ipynb

# flake8: noqa

# set up how Jupyter notebook behaves:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'   # print all outputs
 
# import sys
import os
import os.path as op
# sys.path.append( os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.path.abspath('')))), "babs"  ))

# from babs.core_functions import babs_init
# from babs.cli import *


There are several arguments when calling `babs-init`:

In [5]:
# This notebook only accepts qsiprep or fmriprep (input ds: BIDS)
# ++++++++++++++++++++++++++++++++++
flag_instance = "qsiprep"
type_session = "multi-ses"
# ++++++++++++++++++++++++++++++++++
where_project = "/Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper/data"
where_notebooks = "/Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper/babs/notebooks"

if flag_instance == "fmriprep":
    if type_session == "multi-ses":
        input_ds = op.join(where_project, "j854e")
    elif type_session == "single-ses":
        input_ds = op.join(where_project, "zd9a6")

    project_name = "test_babs_" + type_session + "_fmriprep"
    bidsapp = "fmriprep"

elif flag_instance == "qsiprep":
    project_name = "test_babs_" + type_session + "_qsiprep"
    bidsapp = "qsiprep"
    if type_session == "multi-ses":
        input_ds = op.join(where_project, "j854e")
    elif type_session == "single-ses":
        input_ds = op.join(where_project, "zd9a6")

else:
    raise Exception("`flag_instance` is not QSIPrep or fMRIPrep!")

container_ds = op.join(where_project, "toybidsapp-container-docker")
container_name = bidsapp + "-0-0-0"  # "toybidsapp-0-0-3"
container_config_yaml_file = op.join(where_notebooks, "example_container_" + flag_instance + ".yaml")


Let's pass these arguments into `babs-init` CLI:

In [6]:
!(babs-init \
    --where_project $where_project \
    --project_name $project_name \
    --input "BIDS" $input_ds \
    --container_ds $container_ds \
    --container_name $container_name \
    --container_config_yaml_file $container_config_yaml_file \
    --type_session $type_session \
    --type_system sge)

DataLad version: 0.17.2
nothing to save, working tree clean

project_root of this BABS project: /Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper/data/test_babs_multi-ses_qsiprep
type of data of this BABS project: multi-ses
job scheduling system of this BABS project: sge


Creating `analysis` folder (also a datalad dataset)...
untracked: .gitignore (file)
  deleted: code/sub_missing_any_ses_required_file.csv (file)
Folder 'analysis' exists in the `project_root` and is a datalad dataset; not to re-create it.

Creating output and input RIA...

Registering the input dataset(s)...
The input dataset #1 'BIDS' has been copied into `analysis` folder; not to copy again.

Checking whether each input dataset is a zipped or unzipped dataset...
input dataset 'BIDS' is considered as an unzipped dataset.
Performing sanity check for any unzipped input dataset...

Adding the container as a sub-dataset of `analysis` dataset...
The container has been added as a sub-dataset; not to do it 